In [1]:
# Import libraries
import numpy as np
import pandas as pd
import re
import nltk
from IPython.display import display
from wordcloud import WordCloud, ImageColorGenerator
import matplotlib.pyplot as plt
from PIL import Image

# Activate the necessary magics
%matplotlib inline
%config InlineBackend.figure_format ='retina'

In [2]:
# Setting display options
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', None)

In [31]:
# Loading the dataset spam.csv
#encoding remark: "ISO-8859-1" = "latin-1"
spam_collection = pd.read_csv('dataset/spam.csv', encoding='ISO-8859-1')
print(spam_collection.shape)
spam_collection.head()

(5572, 5)


,type,text,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives around here though",NaN,NaN,NaN


In [4]:
spam_collection.drop(columns=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], inplace=True)
spam_collection.rename(columns = {'type':'Label','text':'SMS'},inplace=True)
spam_collection.head()

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives around here though"


In [5]:
# Randomize the entire data set
randomized_collection = spam_collection.sample(frac=1, random_state=3)

# Calculate index for split
training_test_index = round(len(randomized_collection) * 0.8)

# Training/Test split
training_set = randomized_collection[:training_test_index].reset_index(drop=True)
test_set = randomized_collection[training_test_index:].reset_index(drop=True)

print(training_set.shape)
print(test_set.shape)

(4458, 2)
(1114, 2)


In [6]:
# Original training set - before processing
training_set.head()

,Label,SMS
0,ham,I WILL CAL YOU SIR. In meeting
1,spam,"Loan for any purpose å£500 - å£75,000. Homeowners + Tenants welcome. Have you been previously refused? We can still help. Call Free 0800 1956669 or text back 'help'"
2,ham,LOOK AT THE FUCKIN TIME. WHAT THE FUCK YOU THINK IS UP
3,ham,"Ever green quote ever told by Jerry in cartoon \A Person Who Irritates u Always Is the one Who Loves u Vry Much But Fails to Express It...!..!! :-) :-) gud nyt"""
4,ham,Wat time Ì_ finish?


In [7]:
# Replace email address with 'emailaddress'
training_set['SMS'] = training_set['SMS'].str.replace(r'^.+@[^\.].*\.[a-z]{2,}$', 'emailaddress')

# Replace urls with 'webaddress'
training_set['SMS'] = training_set['SMS'].str.replace(r'^http\://[a-zA-Z0-9\-\.]+\.[a-zA-Z]{2,3}(/\S*)?$', 'webaddress')

# Replace money symbol with 'money-symbol'
training_set['SMS'] = training_set['SMS'].str.replace(r'£|\$', 'money-symbol')

# Replace 10 digit phone number with 'phone-number'
training_set['SMS'] = training_set['SMS'].str.replace(r'^\(?[\d]{3}\)?[\s-]?[\d]{3}[\s-]?[\d]{4}$', 'phone-number')

# Replace normal number with 'number'
training_set['SMS'] = training_set['SMS'].str.replace(r'\d+(\.\d+)?', 'number')

# remove punctuation
training_set['SMS'] = training_set['SMS'].str.replace(r'[^\w\d\s]', ' ')

# remove underscores
training_set['SMS'] = training_set['SMS'].str.replace(r'_[\w\d\s]', ' ')

# remove whitespace between terms with single space
training_set['SMS'] = training_set['SMS'].str.replace(r'\s+', ' ')

# remove leading and trailing whitespace
training_set['SMS'] = training_set['SMS'].str.replace(r'^\s+|\s*?$', ' ')

# change words to lower case
training_set['SMS'] = training_set['SMS'].str.lower()

C:\Users\mko1.NFK\AppData\Local\Temp/ipykernel_2300/375232504.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  training_set['SMS'] = training_set['SMS'].str.replace(r'^.+@[^\.].*\.[a-z]{2,}$', 'emailaddress')
C:\Users\mko1.NFK\AppData\Local\Temp/ipykernel_2300/375232504.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  training_set['SMS'] = training_set['SMS'].str.replace(r'^http\://[a-zA-Z0-9\-\.]+\.[a-zA-Z]{2,3}(/\S*)?$', 'webaddress')
C:\Users\mko1.NFK\AppData\Local\Temp/ipykernel_2300/375232504.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  training_set['SMS'] = training_set['SMS'].str.replace(r'£|\$', 'money-symbol')
C:\Users\mko1.NFK\AppData\Local\Temp/ipykernel_2300/375232504.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  training_set['SMS'] = training_set['SMS'].str

In [8]:
training_set.head()

,Label,SMS
0,ham,i will cal you sir in meeting
1,spam,loan for any purpose åmoney symbolnumber åmoney symbolnumber number homeowners tenants welcome have you been previously refused we can still help call free number number or text back help
2,ham,look at the fuckin time what the fuck you think is up
3,ham,ever green quote ever told by jerry in cartoon a person who irritates u always is the one who loves u vry much but fails to express it gud nyt
4,ham,wat time ì finish


In [9]:
from nltk.corpus import stopwords
stop_words = nltk.corpus.stopwords.words('english')

training_set['SMS'] = training_set['SMS'].apply(lambda x: ' '.join(
    term for term in x.split() if term not in set(stop_words))
)

training_set.head()

,Label,SMS
0,ham,cal sir meeting
1,spam,loan purpose åmoney symbolnumber åmoney symbolnumber number homeowners tenants welcome previously refused still help call free number number text back help
2,ham,look fuckin time fuck think
3,ham,ever green quote ever told jerry cartoon person irritates u always one loves u vry much fails express gud nyt
4,ham,wat time ì finish


In [10]:
lemmatizer = nltk.stem.WordNetLemmatizer()
training_set['SMS'] = training_set['SMS'].apply(lambda x: ' '.join(
    lemmatizer.lemmatize(term, pos='v') for term in x.split())
)

training_set.head()

,Label,SMS
0,ham,cal sir meet
1,spam,loan purpose åmoney symbolnumber åmoney symbolnumber number homeowners tenant welcome previously refuse still help call free number number text back help
2,ham,look fuckin time fuck think
3,ham,ever green quote ever tell jerry cartoon person irritate u always one love u vry much fail express gud nyt
4,ham,wat time ì finish


In [11]:
ss = nltk.SnowballStemmer("english")
training_set['SMS'] = training_set['SMS'].apply(lambda x: ' '.join(
    ss.stem(term) for term in x.split())
)


training_set.head()

,Label,SMS
0,ham,cal sir meet
1,spam,loan purpos åmoney symbolnumb åmoney symbolnumb number homeown tenant welcom previous refus still help call free number number text back help
2,ham,look fuckin time fuck think
3,ham,ever green quot ever tell jerri cartoon person irrit u alway one love u vri much fail express gud nyt
4,ham,wat time ì finish


In [12]:
training_set['SMS'] = training_set['SMS'].apply(lambda sms: nltk.word_tokenize(sms))

training_set.head()

,Label,SMS
0,ham,"[cal, sir, meet]"
1,spam,"[loan, purpos, åmoney, symbolnumb, åmoney, symbolnumb, number, homeown, tenant, welcom, previous, refus, still, help, call, free, number, number, text, back, help]"
2,ham,"[look, fuckin, time, fuck, think]"
3,ham,"[ever, green, quot, ever, tell, jerri, cartoon, person, irrit, u, alway, one, love, u, vri, much, fail, express, gud, nyt]"
4,ham,"[wat, time, ì, finish]"


In [13]:
corpus = training_set['SMS'].sum()

len(corpus)

42751

In [14]:
# Transform the list to a set, to remove duplicates
temp_set = set(corpus)

# Revert to a list
vocabulary = list(temp_set)

len(vocabulary)

5782

In [15]:
# Create the dictionary
len_training_set = len(training_set['SMS'])
word_counts_per_sms = {unique_word: [0] * len_training_set for unique_word in vocabulary}

for index, sms in enumerate(training_set['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1
        
# Convert to dataframe
word_counts = pd.DataFrame(word_counts_per_sms)

word_counts.head()

garag  thatåõ  spell  purpl  hall  haventcn  dosometh  reach  thinl  stand  \
0      0       0      0      0     0         0         0      0      0      0   
1      0       0      0      0     0         0         0      0      0      0   
2      0       0      0      0     0         0         0      0      0      0   
3      0       0      0      0     0         0         0      0      0      0   
4      0       0      0      0     0         0         0      0      0      0   

   maat  doll  monday  ericson  numbernd  bsn  classmat  saw  drop  wesley  \
0     0     0       0        0         0    0         0    0     0       0   
1     0     0       0        0         0    0         0    0     0       0   
2     0     0       0        0         0    0         0    0     0       0   
3     0     0       0        0         0    0         0    0     0       0   
4     0     0       0        0         0    0         0    0     0       0   

   xchat  honi  fool  ffection  wana  year  ujhhhhhhh  entitl  toyota  lar  \
0      0     0     0         0     0     0          0       0       0    0   
1      0     0     0         0     0     0          0       0       0    0   
2      0     0     0         0     0     0          0       0       0    0   
3      0     0     0         0     0     0          0       0       0    0   
4      0     0     0         0     0     0          0       0       0    0   

   limp  toplay  network  dom  axel  waz  divorc  st  sport  extra  jiu  \
0     0       0        0    0     0    0       0   0      0      0    0   
1     0       0        0    0     0    0       0   0      0      0    0   
2     0       0        0    0     0    0       0   0      0      0    0   
3     0       0        0    0     0    0       0   0      0      0    0   
4     0       0        0    0     0    0       0   0      0      0    0   

   appropri  eckankar  bluray  badrith  timi  mnumber  suffer  ultim  \
0         0         0       0        0     0        0       0      0   
1         0         0       0        0     0        0       0      0   
2         0         0       0        0     0        0       0      0   
3         0         0       0        0     0        0       0      0   
4         0         0       0        0     0        0       0      0   

   sarasota  kavalan  whore  anyhow  fedex  decor  shanghai  float  g  ali  \
0         0        0      0       0      0      0         0      0  0    0   
1         0        0      0       0      0      0         0      0  0    0   
2         0        0      0       0      0      0         0      0  0    0   
3         0        0      0       0      0      0         0      0  0    0   
4         0        0      0       0      0      0         0      0  0    0   

   parantella  babygoodby  chase  suzi  writh  north  numberhour  necessari  \
0           0           0      0     0      0      0           0          0   
1           0           0      0     0      0      0           0          0   
2           0           0      0     0      0      0           0          0   
3           0           0      0     0      0      0           0          0   
4           0           0      0     0      0      0           0          0   

   arestaur  jeri  intrepid  fall  heavi  numberfew  join  simul  eppolum  gm  \
0         0     0         0     0      0          0     0      0        0   0   
1         0     0         0     0      0          0     0      0        0   0   
2         0     0         0     0      0          0     0      0        0   0   
3         0     0         0     0      0          0     0      0        0   0   
4         0     0         0     0      0          0     0      0        0   0   

   funk  aroundn  tiim  bluetooth  turkey  wml  spl  pushbutton  taxi  dec  \
0     0        0     0          0       0    0    0           0     0    0   
1     0        0     0          0       0    0    0           0     0    0   
2     0        0     0       

In [16]:
word_counts.shape

(4458, 5782)

In [17]:
# Concatenate with the original training set
training_set_final = pd.concat([training_set, word_counts], axis=1)

training_set_final.head()

Label  \
0   ham   
1  spam   
2   ham   
3   ham   
4   ham   

                                                                                                                                                                   SMS  \
0                                                                                                                                                     [cal, sir, meet]   
1  [loan, purpos, åmoney, symbolnumb, åmoney, symbolnumb, number, homeown, tenant, welcom, previous, refus, still, help, call, free, number, number, text, back, help]   
2                                                                                                                                    [look, fuckin, time, fuck, think]   
3                                           [ever, green, quot, ever, tell, jerri, cartoon, person, irrit, u, alway, one, love, u, vri, much, fail, express, gud, nyt]   
4                                                                                                                                               [wat, time, ì, finish]   

   garag  thatåõ  spell  purpl  hall  haventcn  dosometh  reach  thinl  stand  \
0      0       0      0      0     0         0         0      0      0      0   
1      0       0      0      0     0         0         0      0      0      0   
2      0       0      0      0     0         0         0      0      0      0   
3      0       0      0      0     0         0         0      0      0      0   
4      0       0      0      0     0         0         0      0      0      0   

   maat  doll  monday  ericson  numbernd  bsn  classmat  saw  drop  wesley  \
0     0     0       0        0         0    0         0    0     0       0   
1     0     0       0        0         0    0         0    0     0       0   
2     0     0       0        0         0    0         0    0     0       0   
3     0     0       0        0         0    0         0    0     0       0   
4     0     0       0        0         0    0         0    0     0       0   

   xchat  honi  fool  ffection  wana  year  ujhhhhhhh  entitl  toyota  lar  \
0      0     0     0         0     0     0          0       0       0    0   
1      0     0     0         0     0     0          0       0       0    0   
2      0     0     0         0     0     0          0       0       0    0   
3      0     0     0         0     0     0          0       0       0    0   
4      0     0     0         0     0     0          0       0       0    0   

   limp  toplay  network  dom  axel  waz  divorc  st  sport  extra  jiu  \
0     0       0        0    0     0    0       0   0      0      0    0   
1     0       0        0    0     0    0       0   0      0      0    0   
2     0       0        0    0     0    0       0   0      0      0    0   
3     0       0        0    0     0    0       0   0      0      0    0   
4     0       0        0    0     0    0       0   0      0      0    0   

   appropri  eckankar  bluray  badrith  timi  mnumber  suffer  ultim  \
0         0         0       0        0     0        0       0      0   
1         0         0       0        0     0        0       0      0   
2         0         0       0        0     0        0       0      0   
3         0         0       0        0     0        0       0      0   
4         0         0       0        0     0        0       0      0   

   sarasota  kavalan  whore  anyhow  fedex  decor  shanghai  float  g  ali  \
0         0        0      0       0      0      0         0      0  0    0   
1         0        0      0       0      0      0         0      0  0    0   
2         0        0      0       0      0      0         0      0  0    0   
3         0        0      0       0      0      0         0      0  0    0   
4         0        0      0       0      0      0         0      0  0    0   

   parantella  babygoodby  chase  suzi  writh  north  numberhour  necessari  \
0           0           0      0     0      0      0     

In [18]:
# Filter the spam and ham dataframes
spam_df = training_set_final[training_set_final['Label'] == 'spam'].copy()
ham_df = training_set_final[training_set_final['Label'] == 'ham'].copy()

# Calculate P(Spam) and P(Ham)
p_spam = spam_df.shape[0] / training_set_final.shape[0]
p_ham = ham_df.shape[0] / training_set_final.shape[0]

print(p_spam)
p_ham

0.13660834454912515


0.8633916554508748

In [19]:
# Calculate Nspam, Nham and Nvocabulary
spam_words_per_message = spam_df['SMS'].apply(len)
n_spam = spam_words_per_message.sum()

ham_words_per_message = ham_df['SMS'].apply(len)
n_ham = ham_words_per_message.sum()

n_vocabulary = len(vocabulary)

In [20]:
# Opting for the Laplace smoothing
alpha = 1

In [21]:
# Create two dictionaries that match each unique word with the respective probability value.
parameters_spam = {unique_word: 0 for unique_word in vocabulary}
parameters_ham = {unique_word: 0 for unique_word in vocabulary}

# Iterate over the vocabulary and for each word, calculate P(wi|Spam) and P(wi|Ham)
for unique_word in vocabulary:
    p_unique_word_spam = (spam_df[unique_word].sum() + alpha) / (n_spam + alpha * n_vocabulary)
    p_unique_word_ham = (ham_df[unique_word].sum() + alpha) / (n_ham + alpha * n_vocabulary)
    
    # Update the calculated propabilities to the dictionaries
    parameters_spam[unique_word] = p_unique_word_spam
    parameters_ham[unique_word] = p_unique_word_ham

In [22]:
def sms_classify(message):
    
    # Replace addresses (hhtp, email), numbers (plain, phone), money symbols
    message = message.replace(r'\b[\w\-.]+?@\w+?\.\w{2,4}\b', ' ')
    message = message.replace(r'(http[s]?\S+)|(\w+\.[A-Za-z]{2,4}\S*)', ' ')
    message = message.replace(r'£|\$', ' ')    
    message = message.replace(r'\b(\+\d{1,2}\s)?\d?[\-(.]?\d{3}\)?[\s.-]?\d{3}[\s.-]?\d{4}\b', ' ')    
    message = message.replace(r'\d+(\.\d+)?', ' ')

    # Remove punctuation, collapse all whitespace (spaces, line breaks, tabs) into a single space & eliminate any leading/trailing whitespace.
    message = message.replace(r'[^\w\d\s]', ' ')
    message = message.replace(r'\s+', ' ')
    message = message.replace(r'^\s+|\s+?$', '')

    # Lowercase the entire corpus
    message = message.lower()

    # Remove stop words    
    terms = []
    for term in message.split():
        if term not in set(stop_words):
            terms.append(term)
            message = ' '.join(terms)

    # Lemmatization
    message = ' '.join(lemmatizer.lemmatize(term, pos='v') for term in message.split())            
            
    # Stemming
    message = ' '.join(ss.stem(term) for term in message.split())  
    
    # Tokenization
    message = message.split()
    
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]
    
        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]
    
    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)

    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal probabilities ~ Human action needed!')

In [23]:
sms_classify('''Hey, Sign up with this promo code and get your card for amazing
                exchange fees abroad and £5 to spend anywhere! Promocode: D48KV7BN''')

P(Spam|message): 2.2227632754858812e-34
P(Ham|message): 1.5497270457905326e-34
Label: Spam


In [24]:
sms_classify('''Okey Stan! Seems to be a reasonable amount of money. I'll think
                of it and let you know ASAP.''')

P(Spam|message): 7.856388206992613e-28
P(Ham|message): 1.2358830412546827e-22
Label: Ham


In [25]:
# We define the classify () function again, this time returning the outcomes
def sms_classify_test_set(message):
    '''
    Takes in as input a new sms (w1, w2, ..., wn),
    calculates P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn),
    compares them and returns the spam or ham label, respectively.
    '''
    
    # Replace addresses (hhtp, email), numbers (plain, phone), money symbols
    message = message.replace(r'\b[\w\-.]+?@\w+?\.\w{2,4}\b', ' ')
    message = message.replace(r'(http[s]?\S+)|(\w+\.[A-Za-z]{2,4}\S*)', ' ')
    message = message.replace(r'£|\$', ' ')    
    message = message.replace(r'\b(\+\d{1,2}\s)?\d?[\-(.]?\d{3}\)?[\s.-]?\d{3}[\s.-]?\d{4}\b', ' ')    
    message = message.replace(r'\d+(\.\d+)?', ' ')

    # Remove punctuation, collapse all whitespace (spaces, line breaks, tabs) into a single space & eliminate any leading/trailing whitespace.
    message = message.replace(r'[^\w\d\s]', ' ')
    message = message.replace(r'\s+', ' ')
    message = message.replace(r'^\s+|\s+?$', '')

    # Lowercase the entire corpus
    message = message.lower()
    
    # Remove stop words    
    terms = []
    for term in message.split():
        if term not in set(stop_words):
            terms.append(term)
            message = ' '.join(terms)
    
    # Lemmatization
    message = ' '.join(lemmatizer.lemmatize(term, pos='v') for term in message.split())
    
    # Stemming
    message = ' '.join(ss.stem(term) for term in message.split())
    
    # Tokenization
    message = message.split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]

        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]

    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_spam_given_message > p_ham_given_message:
        return 'spam'
    else:
        return 'needs human classification'

In [26]:
test_set['sms_predicted'] = test_set['SMS'].apply(sms_classify_test_set)
test_set.head(10)

,Label,SMS,sms_predicted
0,ham,"I have 2 sleeping bags, 1 blanket and paper and phone details. Anything else?",ham
1,ham,The search 4 happiness is 1 of d main sources of unhappiness! Accept life the way it comes! U will find happiness in every moment u live.,ham
2,ham,It took Mr owl 3 licks,ham
3,ham,Nothin comes to my mind. ÌÏ help me buy hanger lor. Ur laptop not heavy?,ham
4,ham,Ok...,ham
5,ham,U can call me now...,ham
6,ham,"Hmmm... Guess we can go 4 kb n power yoga... Haha, dunno we can tahan power yoga anot... Thk got lo oso, forgot liao...",ham
7,spam,Hi its LUCY Hubby at meetins all day Fri & I will B alone at hotel U fancy cumin over? Pls leave msg 2day 09099726395 Lucy x Callså£1/minMobsmoreLKPOBOX177HP51FL,ham
8,ham,Can Ì_ all decide faster cos my sis going home liao..,ham
9,ham,Ok... Help me ask if she's working tmr a not?,ham


In [27]:
# Calculate the accuracy of the algorithm
correct = 0
total = test_set.shape[0]

for row in test_set.iterrows():
    row = row[1]
    if row['Label'] == row['sms_predicted']:
        correct += 1

print('Results \n-------')
print('Valid:', correct)
print('Invalid:', total - correct)
print('Accuracy:', round(correct/total, 4))

Results 
-------
Valid: 1089
Invalid: 25
Accuracy: 0.9776
